# 装饰器



## 从debug版本说起

大家也许经常听说，一个程序有release和debug版本。release是正式版，可执行文件较小，运行速度较快。debug版本是调试版本，可执行文件比较大，代码运行较慢，可以使用单步执行、跟踪等功能。除此之外，debug版本会打印更多信息。

假定我们有一个非常简单的函数，返回两个参数的和。

In [2]:
def add(x, y):
    """return the sum of two argument"""
    return x + y

In [3]:
print(add(1, 2), add(1.0, 2.0))
print(add('1.0', 2.0))

3 3.0


TypeError: must be str, not float

希望有一个debug版本，可以打印出函数输入及其参数，以及返回结果。可以利用Python中内置常量`__debug__`来轻松实现。运行Python时`__debug__`值为`True`，当使用选项`-O`会其值`False`。

In [4]:
import time

def add_debug(x, y):
    """the add debug function"""
    if __debug__:
        print(time.time(), 'add entry...')
        print(time.time(), x, type(x))
        print(time.time(), y, type(y))
        
    result = x + y
    
    if __debug__:
        print(time.time(), result, type(result))
        print(time.time(), 'add exit')
        
    return result

In [6]:
add_debug(1, 2)
add_debug(1.0, 2.0)

1554107138.351714 add entry...
1554107138.3527093 1 <class 'int'>
1554107138.3527093 2 <class 'int'>
1554107138.3527093 3 <class 'int'>
1554107138.353713 add exit
1554107138.353713 add entry...
1554107138.3546538 1.0 <class 'float'>
1554107138.3547242 2.0 <class 'float'>
1554107138.3547242 3.0 <class 'float'>
1554107138.3547242 add exit


3.0

In [7]:
add_debug(1, 'x')

1554107145.1006548 add entry...
1554107145.1006548 1 <class 'int'>
1554107145.1016533 x <class 'str'>


TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [8]:
%%writefile add_debug.py
import time

def add_debug(x, y):
    """the add debug function"""
    if __debug__:
        print(time.time(), 'add entry...')
        print(time.time(), x, type(x))
        print(time.time(), y, type(y))
        
    result = x + y
    
    if __debug__:
        print(time.time(), result, type(result))
        print(time.time(), 'add exit')
        
    return result

add_debug(1, 2)
add_debug(1.0, 2.0)
add_debug(1.0, 'str')

Writing add_debug.py


In [9]:
!python add_debug.py

1554107229.6863425 add entry...
1554107229.6863425 1 <class 'int'>
1554107229.6863425 2 <class 'int'>
1554107229.6863425 3 <class 'int'>
1554107229.6863425 add exit
1554107229.6863425 add entry...
1554107229.6863425 1.0 <class 'float'>
1554107229.6863425 2.0 <class 'float'>
1554107229.6863425 3.0 <class 'float'>
1554107229.6863425 add exit
1554107229.6863425 add entry...
1554107229.6863425 1.0 <class 'float'>
1554107229.6863425 str <class 'str'>


Traceback (most recent call last):
  File "add_debug.py", line 20, in <module>
    add_debug(1.0, 'str')
  File "add_debug.py", line 10, in add_debug
    result = x + y
TypeError: unsupported operand type(s) for +: 'float' and 'str'


In [10]:
!python -O add_debug.py

Traceback (most recent call last):
  File "add_debug.py", line 20, in <module>
    add_debug(1.0, 'str')
  File "add_debug.py", line 10, in add_debug
    result = x + y
TypeError: unsupported operand type(s) for +: 'float' and 'str'


对于一个已知函数，有时需要增加一些新的功能，如这里示例所示要增加一些debug信息。当前这个方法显得很费事。不过，在Python中函数也是对象，也可以作为参数来传递，故可以通过传入已知函数名来实现debug功能。

In [11]:
import time

def add_debug2(func, x, y):
    """the add debug function"""
    if __debug__:
        print(time.time(), 'function {} entry...'.format(func.__name__))
        print(time.time(), x, type(x))
        print(time.time(), y, type(y))
        
    result = func(x, y)
    
    if __debug__:
        print(time.time(), result, type(result))
        print(time.time(), 'function {} exit'.format(func.__name__))
        
    return result

add_debug2(add, 1, 2)
add_debug2(add, 1.0, 2.0)
add_debug2(add, 1.0, 'str')

1554107453.1154244 function add entry...
1554107453.1154244 1 <class 'int'>
1554107453.1154244 2 <class 'int'>
1554107453.1154244 3 <class 'int'>
1554107453.1164215 function add exit
1554107453.1164215 function add entry...
1554107453.1164215 1.0 <class 'float'>
1554107453.1174185 2.0 <class 'float'>
1554107453.1187825 3.0 <class 'float'>
1554107453.1187825 function add exit
1554107453.1187825 function add entry...
1554107453.1187825 1.0 <class 'float'>
1554107453.119413 str <class 'str'>


TypeError: unsupported operand type(s) for +: 'float' and 'str'

尽管这里不用修改原定义的`add`函数，但是由于创建了新的函数，所有调用`add`函数的地方都需要更改。

## 装饰器基础

在Python函数中，可以传入函数参数，还可以定义函数并返回函数对象。下面进一步简化debug版函数。

In [12]:
import time

def add_debug3(func):
    """the add debug function"""
    def new_add(x, y):
        """a new add function"""
        if __debug__:
            print(time.time(), 'function {} entry...'.format(func.__name__))
            print(time.time(), x, type(x))
            print(time.time(), y, type(y))
        
        result = func(x, y)
        
        if __debug__:
            print(time.time(), result, type(result))
            print(time.time(), 'function {} exit'.format(func.__name__))
        
        return result
    
    return new_add

然后，调用函数`add_debug3`，传入`add`函数对象，会返回新的函数对象。

In [13]:
add2 = add_debug3(add)

使用自省的方法，检查这两个函数的差异。

In [14]:
print(type(add), type(add2))
print(add, add2)
print(add.__doc__)
print(add2.__doc__)

<class 'function'> <class 'function'>
<function add at 0x0000024972F120D0> <function add_debug3.<locals>.new_add at 0x0000024972F49048>
return the sum of two argument
a new add function


可以看出返回的`add2`函数仍然是函数，只不过是`add_debug3`中的子函数。可以调用函数。

In [15]:
print(add2(1, 2), add2(1.0, 2.0))

1554107630.4399164 function add entry...
1554107630.4399164 1 <class 'int'>
1554107630.4399164 2 <class 'int'>
1554107630.4399164 3 <class 'int'>
1554107630.4419124 function add exit
1554107630.4419124 function add entry...
1554107630.4419124 1.0 <class 'float'>
1554107630.4419124 2.0 <class 'float'>
1554107630.4429085 3.0 <class 'float'>
1554107630.4439065 function add exit
3 3.0


In [16]:
print(add2('1.0', 2.0))

1554107642.1497366 function add entry...
1554107642.1497366 1.0 <class 'str'>
1554107642.1497366 2.0 <class 'float'>


TypeError: must be str, not float

可以看出，除了函数名字不同外，其它参数调用完全一样。还可以更进一步！

在Python中，函数名是一个变量，是指向一个函数对象。我们可以把`add`变量，指向add_debug3中的子函数。

In [17]:
origin_add = add
add = add_debug3(add)
print(type(origin_add), type(add))
print(origin_add, add)
print(origin_add.__doc__)
print(add.__doc__)

<class 'function'> <class 'function'>
<function add at 0x0000024972F120D0> <function add_debug3.<locals>.new_add at 0x0000024972F121E0>
return the sum of two argument
a new add function


经过这般处理，变量`add`已经是增加了debug功能的新函数了，不过过程还是很艰辛。辛运的是，Python提供了装饰器（Decorator）特性，可以简化上述功能实现。

装饰器语法为：
```
@decorated_function
def function_name(parameters):
    suite
```

基本要素：
- 用`@`引导一个装饰器函数
- 其它不变。

顾名思义，装饰器就是不用修改源代码，就能够为原对象增加新的功能。而装饰器函数的本质，就如同`add_debug3`类似，就是可以传入可调用对象，并且返回可调用对象的函数。而`@decorated_function`语法就相对于执行了如下语句：
```
function_name = decorated_function(function_name)
```

下面使用装饰器语法来重新定义`add`函数。首先定义装饰器函数，其中定义函数`inner(*args, **kwargs)`，以匹配任意调用函数。

In [18]:
import time

def decorated_by_debug(func):
    """the decorated debugdebug function"""
    def inner(*args, **kwargs):
        """a inner function in decorated_debug"""
        if __debug__:
            print(time.time(), 'function {} entry...'.format(func.__name__))
            for arg in args:
                print(time.time(), arg, type(arg))
        
        result = func(*args, **kwargs)
        
        if __debug__:
            print(time.time(), result, type(result))
            print(time.time(), 'function {} exit'.format(func.__name__))
        
        return result
    
    return inner

然后使用装饰器来封装函数

In [19]:
@decorated_by_debug
def add(x, y):
    """return the sum of two argument"""
    return x + y

使用自省方法检查`add`函数对象。

In [20]:
print(type(add), add)
print(add.__doc__)

<class 'function'> <function decorated_by_debug.<locals>.inner at 0x0000024972F49C80>
a inner function in decorated_debug


In [21]:
help(add)

Help on function inner in module __main__:

inner(*args, **kwargs)
    a inner function in decorated_debug



In [22]:
print(add(1, 2), add(1.0, 2.0))

1554107894.050512 function add entry...
1554107894.0515182 1 <class 'int'>
1554107894.0515182 2 <class 'int'>
1554107894.0515182 3 <class 'int'>
1554107894.0525045 function add exit
1554107894.0532842 function add entry...
1554107894.0535023 1.0 <class 'float'>
1554107894.0535023 2.0 <class 'float'>
1554107894.0535023 3.0 <class 'float'>
1554107894.054345 function add exit
3 3.0


如果不想用装饰器进行封装，只需要把`@`语句注释即可。

## 深入装饰器



装饰器有很多种，除了函数的装饰器，也有类的装饰器。